In [1]:
#pragma cling add_library_path("/home/adios/local/mark-vtk-m/lib")
#pragma cling load("vtkm_worklet-1.0")
#pragma cling load("vtkm_cont-1.0")
#pragma cling load("vtkm_rendering-1.0")
#pragma cling add_include_path("/home/adios/local/mark-vtk-m/include/vtkm-1.0/")
#pragma cling add_include_path("/home/adios/local/mark-vtk-m/include/vtkm-1.0/")


#include <lodepng_config_cling.h>
#include <iostream>
#include <string>
#include <VTKmXeusRender.h>

cling::DynamicLibraryManager::loadLibrary(): libvtkm_cont-1.0.so.1: cannot open shared object file: No such file or directory
cling::DynamicLibraryManager::loadLibrary(): libvtkm_worklet-1.0.so.1: cannot open shared object file: No such file or directory


In [2]:
#include <vtkm/cont/testing/MakeTestDataSet.h>
std::unique_ptr<VTKmXeusRender> renderer;


In [3]:
#pragma cling add_library_path("/opt/adios2/lib")
#pragma cling add_library_path("/usr/lib")
#pragma cling add_library_path("/home/mark/local/anaconda3/envs/cling/lib/")
#pragma cling add_include_path("/opt/adios2/include")
#pragma cling add_include_path("/usr/include/mpi")
#pragma cling load("adios2")
#pragma cling load("mpi")
#pragma cling load("mpi_cxx")

#include <adios2.h>
#include <mpi.h>

#include <memory.h>

#include <vtkm/cont/ArrayHandleExtrudeCoords.h>
#include <vtkm/cont/ArrayHandleExtrudeField.h>
#include <vtkm/cont/CellSetExtrude.h>

const int phiMultiplier = 8;
int numNodes, numTris, numPhi, numTimeSteps;

bool running = true;
std::unique_ptr<adios2::IO> fileIO;
std::unique_ptr<adios2::IO> meshIO;
std::unique_ptr<adios2::ADIOS> adios;
std::unique_ptr<adios2::ADIOS> mesh;

std::unique_ptr<adios2::Engine> fileReader, meshReader;



In [4]:

void initializeReaders(std::string meshName)
{
    fileReader->BeginStep(adios2::StepMode::NextAvailable, 0.0f);

    meshReader = std::make_unique<adios2::Engine>(meshIO->Open(meshName, adios2::Mode::Read));
    
    adios2::Variable<int> nVar = meshIO->InquireVariable<int>("n_n");
    adios2::Variable<int> triVar = meshIO->InquireVariable<int>("n_t");
    adios2::Variable<int> phiVar = fileIO->InquireVariable<int>("nphi");

    if (nVar){
        meshReader->Get(nVar, &numNodes, adios2::Mode::Sync);

    }
    if (triVar)
        meshReader->Get(triVar, &numTris, adios2::Mode::Sync);

    if (phiVar){
        fileReader->Get(phiVar, &numPhi, adios2::Mode::Sync);
        std::cout << "phi: " << numPhi << std::endl;
    }
}

In [5]:
vtkm::cont::DataSet readMesh()
{
    std::cout << "numNodes: " << numNodes << ", numTris " << numTris << ", numPhi " << numPhi << std::endl;
   adios2::Variable<double> coordVar = meshIO->InquireVariable<double>("/coordinates/values");

  std::vector<double> buff;

  //const int newPhi = phiMultiplier * numPhi;
    int newPhi = numPhi;

   meshReader->Get(coordVar, buff, adios2::Mode::Sync);

   auto coords = vtkm::cont::make_ArrayHandleExtrudeCoords(buff, newPhi, false);
  std::vector<int> ibuffc, ibuffn;

  //vtkDataArray *conn = NULL, *nextNode = NULL;
  // meshFile->ReadScalarData("/cell_set[0]/node_connect_list", timestate, &conn);
  // if (!meshFile->ReadScalarData("/nextnode", timestate, &nextNode))
  //     meshFile->ReadScalarData("nextnode", timestate, &nextNode);
  auto nodeConnectorVar = meshIO->InquireVariable<int>("/cell_set[0]/node_connect_list");
  auto nextNodeVar = meshIO->InquireVariable<int>("nextnode");
  if (!nodeConnectorVar || !nextNodeVar)
      return vtkm::cont::DataSet();

  meshReader->Get(nodeConnectorVar, ibuffc,adios2::Mode::Sync);
  //auto conn = vtkm::cont::make_ArrayHandle(ibuffc);

  meshReader->Get(nextNodeVar, ibuffn, adios2::Mode::Sync);
  if (ibuffn.size() < 1)
      return vtkm::cont::DataSet();

  //auto nextNode = vtkm::cont::make_ArrayHandle(ibuffn);
  auto cells = vtkm::cont::make_CellSetExtrude(ibuffc, coords, ibuffc);

  vtkm::cont::DataSet ds;
  ds.AddCoordinateSystem(vtkm::cont::CoordinateSystem("coords", coords));
  ds.AddCellSet(cells);

  return ds;

}

In [6]:
vtkm::cont::DataSet 
readValues()
{
    vtkm::cont::DataSet ds;

    auto var = fileIO->InquireVariable<double>("dpot");
    std::vector<double> buff;
    fileReader->Get(var, buff,adios2::Mode::Sync);
    auto dpot = vtkm::cont::make_ArrayHandleExtrudeField(buff, numPhi );

    ds.AddField(vtkm::cont::Field("pointvar", vtkm::cont::Field::Association::POINTS,
                                 buff.size(),
                                 dpot));
    
    return ds;
}

In file included from input_line_12:5:
/home/adios/local/mark-vtk-m/include/vtkm-1.0/vtkm/cont/ArrayHandleExtrudeField.h:50:18: error: no matching constructor for initialization of 'StorageType' (aka
      'Storage<double, internal::StorageTagExtrude>')
  ...StorageType(vtkm::cont::make_ArrayHandle(array, length, copy), numberOfPlanes);
     ^           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/adios/local/mark-vtk-m/include/vtkm-1.0/vtkm/cont/ArrayHandleExtrudeField.h:62:12: note: in instantiation of function template specialization
      'vtkm::cont::make_ArrayHandleExtrudeField<double>' requested here
    return make_ArrayHandleExtrudeField(
           ^
input_line_16:8:29: note: in instantiation of function template specialization
      'vtkm::cont::make_ArrayHandleExtrudeField<double>' requested here
    auto dpot = vtkm::cont::make_ArrayHandleExtrudeField(buff, numPhi );
                            ^
/home/adios/local/mark-vtk-m/include/vtkm-1.0/vtkm

Interpreter Error: 

In [ ]:
void openADIOS()
{
  adios = std::make_unique<adios2::ADIOS>(MPI_COMM_WORLD);
  mesh = std::make_unique<adios2::ADIOS>(MPI_COMM_WORLD);

  int numTimeSteps;

  std::string filename("/home/adios/Tutorial/xgc/totalf_itg_tiny/xgc.3d.bp");
  fileIO = std::make_unique<adios2::IO>(adios->DeclareIO("SST"));
  fileIO->SetEngine("SST");
  meshIO = std::make_unique<adios2::IO>(mesh->DeclareIO("BP"));
  meshIO->SetEngine("BP");

   fileReader = std::make_unique<adios2::Engine>(fileIO->Open(filename, adios2::Mode::Read));
  std::cout << "Open " << filename << std::endl;
  std::cout << __FILE__ << " " << __LINE__ << std::endl;

  const auto variables = fileIO->AvailableVariables();
  std::cout << variables.size() << std::endl;

  for (const auto variablePair : variables) {
    std::cout << "Name: " << variablePair.first;

    for (const auto &parameter : variablePair.second) {
      std::cout << "\t" << parameter.first << ": " << parameter.second << "\n";
    }
  }

}

In [ ]:
void display()
{
        vtkm::cont::DataSet ds;
    initializeReaders("/home/adios/adiosvm/Tutorial/xgc/totalf_itg_tiny/xgc.mesh.bp");
    if (fileReader->BeginStep() == adios2::StepStatus::OK){
        ds = readMesh();        
        fileReader->EndStep();
    }
    try{
        while(fileReader->BeginStep() ==adios2::StepStatus::OK && running){
            ds = readValues();
            std::string fieldNm("pointvar");
            vtkm::rendering::CanvasRayTracer canvas(512,512);
            renderer->Display(ds, canvas, fieldNm);
            fileReader->EndStep();
        }    
        fileReader->Close();
        MPI_Finalize();
    }
    catch(int e){
        fileReader->EndStep();
        fileReader->Close();
        MPI_Finalize();
    }

}

In [ ]:
[](){auto try_and_err = []() {
    
    renderer = std::make_unique<VTKmXeusRender>();
    MPI_Init(NULL,NULL);
    openADIOS();
    //printVars();
    display();
    
};try_and_err();}();

In [ ]:
[](){auto try_and_err = []() {
    renderer = std::make_unique<VTKmXeusRender>();
    MPI_Init(NULL,NULL);
    openADIOS();
    printVars();
    display();
}; try_and_err(); }();